In [14]:
%pip install google-generativeai
%pip install openai
import google.generativeai as genai
import openai
import getpass

Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


In [15]:
print("Enter the model you want to use: openai or gemini")
model_name = input()
print("You selected: ", model_name)
print("Enter your Gemini/openai API key: Please Note this is a demo version, only Gemini and openai is currently supported. Input is hidden")
API_KEY = getpass.getpass()

Enter the model you want to use: openai or gemini
You selected:  openai
Enter your Gemini/openai API key: Please Note this is a demo version, only Gemini and openai is currently supported. Input is hidden


In [16]:
def gemini_chatbot_comm(user_input, history=[]):
    genai.configure(api_key=API_KEY)
    model = genai.GenerativeModel("gemini-1.5-flash")
    chat = model.start_chat(
    history = history
    )
    generation_config = {
        "temperature": 0.1  # Adjust temperature for more deterministic (lower) or more creative (higher) responses
    }
    response = chat.send_message(user_input, generation_config=generation_config)
    history.append({
        "role": "user",
        "parts": [{"text": user_input}]
    })
    history.append({
        "role": "model",
        "parts": [{"text": response.text}]
    })
    return response.text, history

In [17]:
def openai_chatbot_comm(user_input, history=[]):
    openai.api_key = API_KEY
    history.append({
        "role": "user",
        'content': user_input
    })
    
    
    response = openai.chat.completions.create(
        model="gpt-4o-mini", 
        temperature=0.1,
        messages=history
    )
    history.append({
        "role": "assistant",
        'content': response.choices[0].message.content
    })
    return response.choices[0].message.content, history



# response, history = openai_chatbot_comm("Hello")
# print(response)
    

In [18]:
def chatbot_comm(user_input, model, history=[]):
    if 'gemini' in model.lower():
        response, history = gemini_chatbot_comm(user_input, history)
        return response, history
    elif 'openai' in model.lower():
        response, history = openai_chatbot_comm(user_input, history)
        return response, history

# Moderation is not working due to exceeded limit. Will implement later once billing is sorted.
# def moderate_response(response):
#     url = "https://api.openai.com/v1/moderations"
#     client=OpenAI(api_key=api_key)
#     response = client.moderations.create(input=response)
#     return response.results[0].flagged

def collect_info(user_input, model, history=[]):
    sys_message2 = """
    Remember to ask the questions one by one and wait for the user's response before asking the next question.
    Keep asking them questions to understand their needs better. The Following data needs to be collected: 
    1. User's budget 
    2. GPU Intensity 
    3. Multitasking 
    4. Portability
    5. Processing Speed
    6. Display Quality
    The Values for the above parameters should only be Low, Medium, High. Only Budget can be a number in INR. 
    You need to infer the values from the user's input as shown in the example below:
    -- GPU Intensity : High
    -- Multitasking : Medium
    -- Portability : Low
    -- Processing Speed : High
    -- Display Quality : High
    -- Budget : 50000
    Keep asking questions until you have all the information you need. 
    in the response, do not include the above instructions or response.
    If 'Not Done', continue asking questions.
    if 'Done', display the collected information in the above format
    Mandatorily respond with 'Done' or 'Not Done' in the next line after the response
    """
    combined_input = f"{sys_message2}\n\nUser: {user_input}"
    response, history = chatbot_comm(combined_input, model, history)
    #print("bot:", response)
#     print("-"*20)
    while 'Not Done' in response:
        filtered_response_list = response.split('\n')
        #print(filtered_response_list)
        filtered_response = [line for line in filtered_response_list if 'Not Done' not in line]
        response_display = '\n'.join(filtered_response).strip()
        print("bot:", response_display)
        print("-"*80)
        user_input = input()
        print("user:", user_input)
        response, history = chatbot_comm(user_input, model, history)
    #print(response.split('\n')[0])
    return response

# def display_info(data):
#     sys_message3 = """
#     You are the most intelligent laptop recommendation assistant.
#     The information collected from the user is as follows:
#     {data}
#     Use this information to recommend the best laptop for the user.
#     respond with the name of the laptop, its price, and and a small description of the laptop.
#     Respond with three best recommendations.
#     """
#     response, history = chatbot_comm(sys_message3)
#     print("bot:", response)
#     print("-"*20)
    


def user_conversation(model):
    print("Welcome to the Laptop Recommendation Chatbot! Please Note this is a demo version. ")
    sys_message = """You are a helpful assistant that can help user choose the best laptop for their needs. 
    Greet the user and ask how you can help them.
    """
    response, history = chatbot_comm(sys_message, model)
    print("Bot: ", response)
    user_input = input()
    print("You: ", user_input)
    user_req = collect_info(user_input,model, history) 
    return user_req
    #display_info(data)

def convert_laptop_specs(model):
    laptop_specs = """
    1.Tecno Megabook T1

        Price: ₹35,990
        Rating: 67
        Processor: Intel Core i7
        Cores/Threads: 4 Cores / 8 Threads
        RAM: 16 GB
        Primary Storage: 1024 GB SSD
        Secondary Storage: None
        GPU: Integrated Intel GPU
        Display: 15.6-inch, 1920x1080 resolution
        Touchscreen: No
        Operating System: Windows
        Warranty: 1 Year

    2.HP Victus 15

        Price: ₹51,100
        Rating: 73
        Processor: AMD Ryzen 5
        Cores/Threads: 6 Cores / 12 Threads
        RAM: 8 GB
        Primary Storage: 512 GB SSD
        Secondary Storage: None
        GPU: Dedicated AMD GPU
        Display: 15.6-inch, 1920x1080 resolution
        Touchscreen: No
        Operating System: Windows
        Warranty: 1 Year

    3.Acer Extensa 15
        Price: ₹39,990
        Rating: 62
        Processor: Intel Core i5
        Cores/Threads: 12 Cores / 16 Threads
        RAM: 16 GB
        Primary Storage: 512 GB SSD
        Secondary Storage: None
        GPU: Integrated Intel GPU
        Display: 14-inch, 1920x1080 resolution
        Touchscreen: No
        Operating System: Windows
        Warranty: 1 Year
        
    4.Lenovo V15
        Price: ₹28,580
        Rating: 62
        Processor: AMD Ryzen 3
        Cores/Threads: 4 Cores / 8 Threads
        RAM: 8 GB
        Primary Storage: 512 GB SSD
        Secondary Storage: None
        GPU: Integrated AMD GPU
        Display: 15.6-inch, 1920x1080 resolution
        Touchscreen: No
        Operating System: Windows
        Warranty: 1 Year

    5.Apple MacBook Air M1

        Price: ₹70,990
        Rating: 45
        Processor: Apple M1
        Cores/Threads: 8 Cores / 8 Threads
        RAM: 8 GB
        Primary Storage: 256 GB SSD
        Secondary Storage: None
        GPU: Apple Integrated GPU
        Display: 13.3-inch, 2560x1600 resolution
        Touchscreen: No
        Operating System: macOS
        Warranty: 1 Year

    6.Dell Inspiron

        Price: ₹34,990
        Rating: 60
        Processor: Intel Core i3
        Cores/Threads: 6 Cores / 8 Threads
        RAM: 8 GB
        Primary Storage: 512 GB SSD
        Secondary Storage: None
        GPU: Integrated Intel GPU
        Display: 15.6-inch, 1920x1080 resolution
        Touchscreen: No
        Operating System: Windows
        Warranty: 1 Year

    7.Samsung Galaxy Book

        Price: ₹64,990
        Rating: 68
        Processor: Intel Core i5
        Cores/Threads: 12 Cores / 16 Threads
        RAM: 16 GB
        Primary Storage: 512 GB SSD
        Secondary Storage: None
        GPU: Integrated Intel GPU
        Display: 13.3-inch, 1920x1080 resolution
        Touchscreen: No
        Operating System: Windows
        Warranty: 1 Year

    8.MSI Thin GF63

        Price: ₹52,990
        Rating: 71
        Processor: Intel Core i5
        Cores/Threads: 8 Cores / 12 Threads
        RAM: 16 GB
        Primary Storage: 512 GB SSD
        Secondary Storage: None
        GPU: Dedicated Nvidia GPU
        Display: 15.6-inch, 1920x1080 resolution
        Touchscreen: No
        Operating System: Windows
        Warranty: 1 Year

    9.Asus Vivobook

        Price: ₹57,390
        Rating: 71
        Processor: AMD Ryzen 7
        Cores/Threads: 8 Cores / 16 Threads
        RAM: 16 GB
        Primary Storage: 512 GB SSD
        Secondary Storage: None
        GPU: Integrated AMD GPU
        Display: 16-inch, 1920x1200 resolution
        Touchscreen: No
        Operating System: Windows
        Warranty: 1 Year

    10.Lenovo Ideapad

        Price: ₹47,490
        Rating: 66
        Processor: Intel Core i5
        Cores/Threads: 4 Cores / 8 Threads
        RAM: 16 GB
        Primary Storage: 512 GB SSD
        Secondary Storage: None
        GPU: Integrated Intel GPU
        Display: 15.6-inch, 1920x1080 resolution
        Touchscreen: No
        Operating System: Windows
        Warranty: 1 Year

        The above data is the information given by the distributor about laptops they sell. 
        You need to convert this data into a format that can be used to recommend laptops to the user.
        Convert the data into a format given below:
        -- GPU Intensity : High
        -- Multitasking : Medium
        -- Portability : Low
        -- Processing Speed : High
        -- Display Quality : High
        -- Price : 50000

        The Price should be in INR. 
        Once you have converted the data, respond with the converted data.
        """
    response, history = chatbot_comm(laptop_specs, model)
    # print("bot:", response)
    # print("-"*20)
    return response

def compare_laptop(user_req, model, laptop_specs):
    sys_message4 = f"""
    You are the most intelligent laptop recommendation assistant.
    The user requirement information collected from the user is as follows:
    {user_req}
    The information about the laptops is as follows:
    {laptop_specs}
    Use this information to recommend the best laptop that match the {user_req} for the user.
    respond with the name of the laptop, its price, and and a small description of the laptop.
    Respond with 1 top recommendation.
    For the Recommended product, write a good, detailed description which should be creative 
    and unique also should try to sell the product to the customer.
    If No laptops are available in the given budget, respond with "No laptops available in the given budget."
    If No laptops match the user's requirements, respond with "No laptops match the user's requirements currently."
    """
    response, history = chatbot_comm(sys_message4, model)
    print("bot:", response)
    print("-"*20)

user_req = user_conversation(model_name)
laptop_specs = convert_laptop_specs(model_name)
compare_laptop(user_req, model_name, laptop_specs)



Welcome to the Laptop Recommendation Chatbot! Please Note this is a demo version. 
Bot:  Hello! I'm here to help you find the perfect laptop for your needs. What specific tasks or activities do you plan to use the laptop for? Let me know how I can assist you!
You:  I am looking for a gaming laptop
bot: Based on your interest in gaming laptops and considering the specifications and budget, I recommend the **HP Victus 15**.

**Price:** ₹51,100

**Description:**  
The HP Victus 15 is a powerhouse designed specifically for gamers who crave performance without breaking the bank. Equipped with an AMD Ryzen 5 processor and a dedicated AMD GPU, this laptop ensures smooth gameplay and stunning graphics, making it perfect for both casual and competitive gaming. With 8 GB of RAM, multitasking is a breeze, allowing you to run your favorite games alongside streaming or browsing without any hiccups.

The 15.6-inch Full HD display brings your games to life with vibrant colors and sharp details, immer